In [14]:
import sys
sys.path.append(r'C:/Users/andrej/Projects/XRPL/')

In [15]:
import cfRestApiV3 as cfApi
import datetime
import pandas as pd
import json
import matplotlib.pyplot as plt
import os

In [16]:
from Utilities.email_sending import send_plain_email, send_html_email

EMAIL_PASSWORD = os.getenv('EMAIL_PASSWORD') # the password needs to be set as EMAIL_PASSWORD in system variables of the computer where the process is running, it is located in S:/Algo/email_password.txt
if EMAIL_PASSWORD is None:
    raise ValueError("EMAIL_PASSWORD system variable not set")

RECIPIENT = ["azubal@hotmail.com"] # can also be a list of recipients

In [17]:
def convert_to_html_content(input_string):
    # Split the input string by the newline character
    lines = input_string.split('\n')
    # Wrap each line in a <p> tag
    html_lines = [f"<p>{line}</p>" for line in lines]
    # Join the lines into a single HTML string
    html_content = '\n'.join(html_lines)
    return html_content


def convert_dataframe_to_html_content(df):
    # Convert the DataFrame to an HTML table
    html_content = '<table border="1" cellpadding="5" cellspacing="0" style="border-collapse: collapse;">\n'

    # Add the header row
    name = 'Order Book'
    html_content += '  <tr>\n'
    html_content += f'    <th>{name}</th>\n'
    for column in df.columns:
        html_content += f'    <th>{column}</th>\n'
    html_content += '  </tr>\n'

    # Add the data rows
    for idx, row in df.iterrows():
        html_content += '  <tr>\n'
        html_content += f'    <td>{idx}</td>\n'  # Add the index as the first column
        for item in row:
            html_content += f'    <td>{item}</td>\n'
        html_content += '  </tr>\n'

    # Close the HTML table
    html_content += '</table>'

    return html_content

In [18]:
# accessible on your Account page under Settings -> API Keys
apiPublicKey = "X0J+mG76qUHLfr7G4Yk0/cSX7+T7cApznHo2opaEOOqNZKed2AmDFuMI"
# accessible on your Account page under Settings -> API Keys
apiPrivateKey = "YMyZG6GhXNmvW/ZvLzFp5zxTX+dmfPJCp8Vzamz8LJ+CBmQmloF7yBC3/sD3mNrMw8i8CtNqMnd9233v3+6kpUlT"

In [19]:
# use "api.cryptofacilities.com" if your IP is whitelisted (Settings -> API Keys -> IP Whitelist)
apiPath = "https://www.cryptofacilities.com"
timeout = 20
checkCertificate = True  # when using the test environment, this must be set to "False"
useNonce = False  # nonce is optional

cfPublic = cfApi.cfApiMethods(
    apiPath, timeout=timeout, checkCertificate=checkCertificate)
cfPrivate = cfApi.cfApiMethods(apiPath, timeout=timeout, apiPublicKey=apiPublicKey,
                               apiPrivateKey=apiPrivateKey, checkCertificate=checkCertificate, useNonce=useNonce)

# XRP FUTURE PRICE ON KRAKEN

In [20]:
# # get prices
symbol='PI_XRPUSD'
tickers=json.loads(cfPublic.get_tickers())['tickers'] 
result =[(x['symbol'], x.get('last', None), x.get('bid', None),  x.get('ask', None)) for x in tickers if x['symbol']==symbol]
mid_price=(result[0][3]+result[0][2])/2
print(mid_price)
pd.DataFrame(result)

2.4395499999999997


,0,1,2,3
0,PI_XRPUSD,2.4436,2.4389,2.4402


# XRP LEDGER ORDERBOOK

In [21]:
import asyncio
from collections import defaultdict
from itertools import accumulate
from xrpl.asyncio.clients import AsyncJsonRpcClient
from xrpl.models.requests import BookOffers

# Define the XRP Ledger JSON-RPC endpoint
XRP_LEDGER_RPC = "https://xrplcluster.com"  # Public XRP Ledger Full-History Server

# Create an async XRP Ledger API client
client = AsyncJsonRpcClient(XRP_LEDGER_RPC)

# Define the market pair: XRP/USD (Bitstamp Issuer)
USD_ISSUER = "rvYAfWj5gh67oV6fW32ZzP3Aw4Eubs59B"  # Bitstamp USD issuer
USD_CURRENCY = "USD"

async def fetch_order_book():
    """ Fetches and displays separate bid and ask offers with cumulative sums. """
    
    # Request bid orders (buy orders) → Users want to BUY XRP, paying USD
    bid_request = BookOffers(
        taker_gets={"currency": "XRP"},  # Buyers receive XRP
        taker_pays={"currency": USD_CURRENCY, "issuer": USD_ISSUER},  # Buyers pay USD
        limit=50
    )

    # Request ask orders (sell orders) → Users want to SELL XRP, receiving USD
    ask_request = BookOffers(
        taker_gets={"currency": USD_CURRENCY, "issuer": USD_ISSUER},  # Sellers receive USD
        taker_pays={"currency": "XRP"},  # Sellers pay XRP
        limit=50
    )

    # Fetch order books concurrently
    bid_response, ask_response = await asyncio.gather(
        client.request(bid_request), client.request(ask_request)
    )

    bid_dict = defaultdict(float)  # Bids (Price -> Total XRP Amount)
    ask_dict = defaultdict(float)  # Asks (Price -> Total XRP Amount)

    # Parse bid orders
    for offer in bid_response.result["offers"]:
        price = round(float(offer["TakerPays"]["value"]) / (float(offer["TakerGets"])/1000000), 6)
        bid_dict[price] += (float(offer["TakerGets"])/1000000)  # Sum XRP amounts

    # Parse ask orders
    for offer in ask_response.result["offers"]:
        price = round(float(offer["TakerGets"]["value"]) / (float(offer["TakerPays"])/1000000), 6)
        ask_dict[price] += (float(offer["TakerPays"])/1000000)  # Sum XRP amounts

    # Convert to sorted lists
    bid_list = sorted(bid_dict.items(), key=lambda x: x[0], reverse=False)[:5]  # Highest bids first
    ask_list = sorted(ask_dict.items(), key=lambda x: x[0], reverse=True)[:5]  # Lowest asks first

    # Compute cumulative sums
    bid_cumsum = list(accumulate(bid[1] for bid in bid_list))
    ask_cumsum = list(accumulate(ask[1] for ask in ask_list))

    # Print order book with bids on left, asks on right
    print(mid_price)
    print("\n🔹 **ORDER BOOK (Bids on Left, Asks on Right) 🔸")
    print(f"{'BID Amount (XRP)':<15} {'Cumulative XRP':<15} {'BID Price (USD/XRP)':<18} || {'ASK Price (USD/XRP)':<18} {'Cumulative XRP':<15} {'ASK Amount (XRP)':<15}")
    print("=" * 85)

    bid_result=[]
    ask_result=[]

    for i in range(max(len(ask_list), len(bid_list))):
        bid_amount = f"{ask_list[i][1]:.2f}" if i < len(ask_list) else ""
        bid_cumulative = f"{ask_cumsum[i]:.2f}" if i < len(ask_cumsum) else ""
        bid_price = f"{ask_list[i][0]:.6f}" if i < len(ask_list) else ""

        ask_price = f"{bid_list[i][0]:.6f}" if i < len(bid_list) else ""
        ask_cumulative = f"{bid_cumsum[i]:.2f}" if i < len(bid_cumsum) else ""
        ask_amount = f"{bid_list[i][1]:.2f}" if i < len(bid_list) else ""

        print(f"{bid_amount:<15} {bid_cumulative:<15} {bid_price:<18} || {ask_price:<18} {ask_cumulative:<15} {ask_amount:<15}")
        
        bid_result.append((bid_price, bid_amount, bid_cumulative))
        ask_result.append((ask_price, ask_amount, ask_cumulative))

    return bid_result, ask_result

# Run the async function in Jupyter Notebook
bids, asks=await fetch_order_book()


2.4395499999999997

🔹 **ORDER BOOK (Bids on Left, Asks on Right) 🔸
BID Amount (XRP) Cumulative XRP  BID Price (USD/XRP) || ASK Price (USD/XRP) Cumulative XRP  ASK Amount (XRP)
100.00          100.00          2.426200           || 2.435110           2.97            2.97           
206138.86       206238.86       2.376018           || 2.446000           103.97          101.00         
101.00          206339.86       2.260000           || 2.448100           203.97          100.00         
200.00          206539.86       2.250000           || 2.477041           48812.79        48608.82       
74.19           206614.05       2.221000           || 2.490000           48912.79        100.00         


# Calculating opportunity

In [22]:
best_bid=max([float(x[0]) for x in bids])
best_ask=min([float(x[0]) for x in asks])

In [23]:
bid_opportunity=0
if mid_price>0.997*best_bid:
    pass
else:
    for bid in bids:
        if mid_price<=bid[0]:
            bid_opportunity+=(0.997*bid[0]-mid_price)*bid[1]
print(bid_opportunity)            

0


In [24]:
ask_opportunity=0
if mid_price<1.003*best_ask:
    pass
else:
    for ask in asks:
        if mid_price>=ask[0]:
            ask_opportunity+=(mid_price-1.003*ask[0])*ask[1]
print(ask_opportunity)            

0


In [25]:
from datetime import datetime

def log_opportunity(filename, opportunity_value):
    """Appends a timestamp and opportunity value to a text file."""
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(filename, "a") as file:
        file.write(f"{timestamp}, {opportunity_value}\n")

# Example usage
log_opportunity(r'C:/Users/andrej/Projects/XRPL/opportunities_log.txt', bid_opportunity+ask_opportunity)  # Replace with actual opportunity value

# Sending email

In [26]:
html_content = f"""\
<html>
    <body>
        
        <h1>The opportunity is high</h1>
        <p>Opportunity: { bid_opportunity+ask_opportunity}</p> 

    </body>
</html>
"""


if bid_opportunity+ask_opportunity>=100:
    send_html_email(
        RECIPIENT, 
        f"XRP opportunity HIGH alert! {bid_opportunity+ask_opportunity}", "", html_content, "",
            email_password=EMAIL_PASSWORD

            )